#  Ventas de Ecommerce Data Mining

In [7]:
!pip install missingno
import pandas as pd
import numpy as np
import missingno as msno
import plotly.express as px
import plotly.graph_objects as go

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

## Obteniendo la información

In [6]:
df = pd.read_csv('./data/sales.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## Entendiendo la información

In [8]:
df.shape

(541909, 8)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [12]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


## Limpieza de datos

In [20]:
# Datos faltantes
df_msn = pd.DataFrame(df.isna().sum())
df_msn['Features'] = df_msn.index
df_msn['MissingValues'] = df_msn[0]
fig = px.bar(df_msn, 
            title='Datos faltantes',
             x = 'Features', 
             y = 'MissingValues', 
             color='Features',
            text='MissingValues',
            text_auto='.2s')
fig.update_yaxes(title='Conteo de datos faltantes')

In [21]:
customer_msn = (df['CustomerID'].isna().sum() / len(df['CustomerID'])) * 100
description_msn = (df['Description'].isna().sum() / len(df['CustomerID'])) * 100
print('{:.2f}%, {:.2f}%'.format(customer_msn,description_msn))

24.93%, 0.27%


- Aproximadamente 140k transacciones no cuentan con la información del cliente. Esto equivale a casi 25% de toda la información. 
- 1.5k elementos tienen dependencias faltantes. Eso equivale al 0.27%

In [22]:
df[df['Description'].isna()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,12/1/2010 14:32,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,12/1/2010 14:34,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535322,581199,84581,NaN,-2,12/7/2011 18:26,0.0,NaN,United Kingdom
535326,581203,23406,NaN,15,12/7/2011 18:31,0.0,NaN,United Kingdom
535332,581209,21620,NaN,6,12/7/2011 18:35,0.0,NaN,United Kingdom
536981,581234,72817,NaN,27,12/8/2011 10:33,0.0,NaN,United Kingdom


- Aqui podemos ver donde la 'Descripcion', 'UnitPrice' y 'CustomerID' son 'NaN'
- Lo mejor es no contar con esa información para el análisis

In [23]:
# Limpiamos NaN de CustomerID y Description
df.drop(df[df['CustomerID'].isna() | df['Description'].isna()].index, inplace=True)

In [24]:
# Limpiamos los elementos que no cuentan con paises
df.drop(df.query('Country == "Unspecified"').index, inplace=True)

In [25]:
df.shape

(406585, 8)

- Ahora tenemos que corregir los tipos de dato de 'InvoiceDate' y 'CustomerID'

In [26]:
df = df.astype({'InvoiceDate':'datetime64[ns]','CustomerID':'int'})
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406585 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406585 non-null  object        
 1   StockCode    406585 non-null  object        
 2   Description  406585 non-null  object        
 3   Quantity     406585 non-null  int64         
 4   InvoiceDate  406585 non-null  datetime64[ns]
 5   UnitPrice    406585 non-null  float64       
 6   CustomerID   406585 non-null  int64         
 7   Country      406585 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 27.9+ MB


- Al revisar la descripcion del dataset (df.describe()) podemos encontrar valores negativos en 'Quantity'
- Tenemos que investigar que significan esos valores

In [27]:
df.query('Quantity < 0')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315,United Kingdom


- Podemos observar que 'InvoiceNo' empieza con una 'C' para los elementos con 'Quantity' negativa
- Podemos asumir que son ordened Canceladas

- Ahora eliminamos información que no nos sirve en el análisis, como:

In [28]:
df.drop(df.query('Description in ["POSTAGE", "CARRIAGE", "Discount", "DOTCOM POSTAGE", "CRUK Commission", "Manual"]').index, axis=0, inplace=True)

- Buscando irregularidades

In [29]:
df.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Greece', 'Singapore', 'Lebanon',
       'United Arab Emirates', 'Saudi Arabia', 'Czech Republic', 'Canada',
       'Brazil', 'USA', 'European Community', 'Bahrain', 'Malta', 'RSA'],
      dtype=object)

- Podemos observar que 'EIRE', 'USA' y 'RSA' son irregulares al resto
- Podemos reemplazarlas por valores más específicos

In [30]:
df.loc[df.query('Country == "EIRE"').index, 'Country'] = 'Ireland'
df.loc[df.query('Country == "USA"').index, 'Country'] = 'United States of America'
df.loc[df.query('Country == "RSA"').index, 'Country'] = 'South Africa'

df.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'Ireland', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Greece', 'Singapore', 'Lebanon',
       'United Arab Emirates', 'Saudi Arabia', 'Czech Republic', 'Canada',
       'Brazil', 'United States of America', 'European Community',
       'Bahrain', 'Malta', 'South Africa'], dtype=object)

## Feature Engineering
- Agregando la columna 'Revenue'

In [31]:
df['Revenue'] = abs(df['UnitPrice'] * df['Quantity'])
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Revenue'],
      dtype='object')

- Obtenemos los elementos de la fecha 'Invoice Data' para mejor análisis

In [32]:
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month_name()
df['Date'] = df[['Month','Year']].astype(str).apply('-'.join, axis=1)
df['Day'] = df['InvoiceDate'].dt.day_name()
df['Hour'] = df['InvoiceDate'].dt.hour

df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,Year,Month,Date,Day,Hour
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,2010,December,December-2010,Wednesday,8
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010,December,December-2010,Wednesday,8
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,2010,December,December-2010,Wednesday,8
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010,December,December-2010,Wednesday,8
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010,December,December-2010,Wednesday,8


In [34]:
def formatTime(h):
    return h.Hour+':00'

## Análisis

In [35]:
# contantes
CURRENCY = '£'
TICKANGLE = 30

In [36]:
# Dataset sin ordenes canceladas
Sales = df[~df['InvoiceNo'].str.contains('C')]
Sales.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,Year,Month,Date,Day,Hour
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,2010,December,December-2010,Wednesday,8
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010,December,December-2010,Wednesday,8
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,2010,December,December-2010,Wednesday,8
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010,December,December-2010,Wednesday,8
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010,December,December-2010,Wednesday,8


- Ahora si podemos obtener las ganancias totales en el dataset

In [41]:
total_revenue = Sales.groupby('Date', as_index=False)['Revenue'].sum().round(2)
order_list = ["05","09","01","13","03","02","08","07","04","06","12","11","10"]
total_revenue['Order'] = order_list
total_revenue.sort_values(by='Order', inplace=True)
total_revenue.reset_index(drop=True, inplace=True)
total_revenue.drop('Order', axis=1, inplace=True)

fig = go.Figure()
fig.add_traces(go.Scatter(x=total_revenue.Date,y=total_revenue.Revenue,line=dict(color='crimson')))
fig.update_traces(name='Ganancias')
fig.update_layout(title='Total de ganancias')
fig.update_xaxes(tickangle=TICKANGLE)
fig.update_yaxes(tickprefix=CURRENCY, title='Ganancias')

fig.show()

total_revenue

,Date,Revenue
0,December-2010,567505.72
1,January-2011,564041.64
2,February-2011,443346.02
3,March-2011,584562.85
4,April-2011,454982.81
5,May-2011,659644.22
6,June-2011,654246.78
7,July-2011,591933.42
8,August-2011,636276.21
9,September-2011,940930.81


- Mes con mayores ganancias: Noviembre 2011 con 1.15m
- Mes con menores ganancias: Febrero 2011 con 443.77k

In [39]:
country_revenue = Sales.groupby('Country', as_index=False)['Revenue'].sum().round(2)
country_revenue.sort_values(by=['Revenue'], ascending=False, inplace=True)

fig = px.bar(country_revenue[:10].reset_index(),
             title='Top 10 paises con mayores ventas',
             x='Country',
             y='Revenue',
             text='Revenue',
             text_auto='.3s',
             color='Country')
fig.update_yaxes(title='Revenue', tickprefix=CURRENCY)
fig.show()

country_revenue[:10]

,Country,Revenue
34,United Kingdom,7266027.23
23,Netherlands,283889.34
16,Ireland,257296.56
13,Germany,205569.89
12,France,183891.68
0,Australia,138171.31
30,Spain,55725.11
32,Switzerland,52441.95
19,Japan,37416.37
3,Belgium,36927.34


In [42]:
products_revenue = Sales.groupby(['StockCode','Description'], as_index=False)['Revenue'] \
.sum().round(2) \
.sort_values('Revenue', ascending=False) \
.head(20) \
.reset_index(drop=True)

fig = px.bar(products_revenue,
            title='Top 20 Productos más vendidos',
            x='StockCode',
            y='Revenue',
            color='StockCode',
            text='Revenue',
            text_auto='.4s')
fig.update_yaxes(tickprefix=CURRENCY)
fig.show()

In [43]:
product_sales = Sales.groupby('StockCode', as_index=False)['Quantity'].sum()
product_sales.rename(columns={'Quantity':'Sales'}, inplace=True)

product_sales_perf = products_revenue.merge(product_sales, on='StockCode')

fig = px.bar(product_sales_perf.reset_index(),
            title='Cantidad de Ventas de los Top 20 Productos',
            x='StockCode',
            y='Sales',
            text='StockCode',
            color='StockCode',
            text_auto='.2s')
fig.update_yaxes(title='Quantity')
fig.show()

In [44]:
unitPrice_grouped = Sales.groupby(['StockCode'], as_index=False)['UnitPrice'].mean()
unit_price = product_sales_perf.merge(unitPrice_grouped, on='StockCode')

fig = px.bar(unit_price,
            title='Precio Promedio de Venta de los Top 20 Productos',
            x='StockCode',
            y='UnitPrice',
            color='StockCode',
            text='UnitPrice',
            text_auto='.2f')
fig.update_yaxes(tickprefix = '£', title='Unit Price')
fig.show()

unit_price

,StockCode,Description,Revenue,Sales,UnitPrice
0,23843,"PAPER CRAFT , LITTLE BIRDIE",168469.60,80995,2.080000
1,22423,REGENCY CAKESTAND 3 TIER,142567.45,12410,12.475842
2,85123A,WHITE HANGING HEART T-LIGHT HOLDER,100448.15,36782,2.893106
3,85099B,JUMBO BAG RED RETROSPOT,85220.78,46181,2.015878
4,23166,MEDIUM CERAMIC TOP STORAGE JAR,81416.73,77916,1.220303
5,47566,PARTY BUNTING,68844.33,15295,4.872885
6,84879,ASSORTED COLOUR BIRD ORNAMENT,56543.16,35340,1.680776
7,23084,RABBIT NIGHT LIGHT,51346.20,27202,2.013943
8,79321,CHILLI LIGHTS,46286.51,9650,5.428760
9,22086,PAPER CHAIN KIT 50'S CHRISTMAS,42660.83,15617,2.937203


In [45]:
top_customers = Sales.groupby('CustomerID', as_index=False)['Revenue'].sum() \
.sort_values(by='Revenue', ascending=False) \
.reset_index(drop=True)
top_customers['CustomerID'] = top_customers['CustomerID'].astype(str)

fig = px.bar(top_customers[:10],
            title='Top 10 Clientes por compra',
            x='CustomerID',
            y='Revenue',
            color='CustomerID',
            text='Revenue',
            text_auto='.4s')
fig.update_yaxes(tickprefix=CURRENCY)
fig.show()

top_customers[:10]

,CustomerID,Revenue
0,14646,279138.02
1,18102,259657.30
2,17450,194550.79
3,16446,168472.50
4,14911,136275.72
5,12415,124564.53
6,14156,116729.63
7,17511,91062.38
8,12346,77183.60
9,16029,72882.09


In [47]:
daily = Sales.groupby('Day', as_index=False)['Revenue'].sum()
daily['Order'] = ['5','1','6','4','2','3']
daily.sort_values(by=['Order'], inplace=True)

fig = px.bar(daily.reset_index(),
            title='Ventas por dia de la semana',
            x='Day',
            y='Revenue',
            text='Day',
            color='Day',
            text_auto='.2s')
fig.update_yaxes(tickprefix = '£', title='Revenue')
fig.show()

In [48]:
product_sales = Sales.groupby('Hour', as_index=False)['Revenue'].sum()
product_sales['Hour'] = product_sales['Hour'].astype(str)
product_sales['Hour'] = product_sales.apply(lambda h:formatTime(h), axis=1)

fig = px.line(product_sales,
            title='Ventas por hora del dia',
            x='Hour',
            y='Revenue')
fig.update_yaxes(tickprefix = '£', title='Unit Price')
fig.show()

In [50]:
products_canceled = df.query('(InvoiceNo.str.contains("C"))') \
.groupby(['Date'], as_index=False) \
.Revenue.agg(['count','sum']) \
.reset_index()

products_canceled.rename(columns={'count':'CanceledOrders','sum':'RevenueLost'}, inplace=True)
order_list = ["05","09","01","13","03","02","08","07","04","06","12","11","10"]
products_canceled['Order'] = order_list
products_canceled.sort_values(by='Order', ascending=True, inplace=True)
products_canceled.reset_index(drop=True, inplace=True)
products_canceled.drop('Order', axis=1,inplace=True)

fig = go.Figure()
fig.add_traces(go.Scatter(name='RevenueLost',x=products_canceled.Date,y=products_canceled.RevenueLost,line=dict(color='crimson')))
fig.update_layout(title='Dinero perdido por ordenes canceladas')
fig.update_yaxes(tickprefix=CURRENCY, title='Revenue')
fig.show()

# Conclusiones y conocimiento adquirido

- En septiembre, 944,4 mil libras esterlinas, hubo un aumento significativo en los ingresos en comparación con el mes anterior de agosto (639,3 mil libras esterlinas), lo que representa un crecimiento de aproximadamente el 47,8%. Esta tendencia ascendente continuó de manera constante hasta noviembre de 2011, que registró los mayores ingresos por ventas de £1.145,2 mil desde diciembre de 2010 a diciembre de 2011. Sin embargo, diciembre de 2011 experimentó una caída del 20,4% en los ingresos a £513,7 mil. Es necesario realizar más investigaciones para determinar las causas detrás del aumento de los ingresos y la fuerte caída.
---
- El Reino Unido generó los ingresos más altos de £7,29 millones por ventas entre diciembre de 2010 y diciembre de 2011, lo que es más de un 2000% más que el siguiente mejor desempeño con £283,9 mil. Esto indica la necesidad de esfuerzos de marketing adicionales en otros países.
---
- Entre diciembre de 2010 y diciembre de 2011, el producto con mayores ingresos fue "PAPER CRAFT, LITTLE BIRDIE", que generó unos ingresos de £168,5 mil.
---
- Los productos "REGENCY CAKESTAND 3 TIER" y "PICNIC BASKET WICKER 60 PIECES" tuvieron precios unitarios por producto relativamente altos, £12,48 y £11,20 respectivamente. Estos precios deberían revisarse.
---
- El cliente con ID 14646 realizó el mayor número de compras entre 2010 y 2011, por valor de 279,1 mil libras esterlinas.
---
- Los clientes tienden a realizar más compras los jueves, siendo las horas pico entre las 10:00 a. m. y las 12:00 p. m. Los equipos de ventas y marketing deben orientar sus esfuerzos en consecuencia.